# Data Preprocessing

## Importing Libraries

In [1]:
#importing Libraries
import pandas as pd
import os
import sys

#import local libraries
#Adding scripts path
sys.path.append(os.path.abspath(os.path.join('..')))
#importing dvc_data_loader script
# from scripts.dvc_data_loader import *
from scripts.data_information import DataInfo
from scripts.data_loader import load_df_from_csv
from scripts.data_manipulation import DataManipulator
from scripts.data_cleaner import DataCleaner
from scripts.utlity_functions import convert_to_month_name
from scripts.grapher import *
from scripts.dvc_data_loader import get_dvc_data
from sklearn import preprocessing

In [2]:
pd.set_option('max_column', None)


## Loading Data

In [3]:
combined_df = load_df_from_csv('../data/train.csv')
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 22 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Unnamed: 0                 1017209 non-null  uint32 
 1   Store                      1017209 non-null  uint16 
 2   DayOfWeek                  1017209 non-null  uint8  
 3   Date                       1017209 non-null  object 
 4   Year                       1017209 non-null  uint16 
 5   Month                      1017209 non-null  uint8  
 6   Day                        1017209 non-null  uint8  
 7   Sales                      1017209 non-null  uint16 
 8   Customers                  1017209 non-null  uint16 
 9   Open                       1017209 non-null  uint8  
 10  Promo                      1017209 non-null  uint8  
 11  StateHoliday               1017209 non-null  object 
 12  SchoolHoliday              1017209 non-null  uint8  
 13  StoreType   

In [4]:
combined_df.head(5)

,Unnamed: 0,Store,DayOfWeek,Date,Year,Month,Day,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,0,1,5,2015-07-31,2015,7,31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,1,4,2015-07-30,2015,7,30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,2,1,3,2015-07-29,2015,7,29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,3,1,2,2015-07-28,2015,7,28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,4,1,1,2015-07-27,2015,7,27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [5]:
combined_df.drop(['Unnamed: 0'], axis=1, inplace=True)

## Handling None and Empty Values

In [6]:
info = DataInfo(combined_df)
info.get_column_based_missing_percentage()

,total_missing_values,missing_percentage
Store,0,0.0 %
DayOfWeek,0,0.0 %
Date,0,0.0 %
Year,0,0.0 %
Month,0,0.0 %
Day,0,0.0 %
Sales,0,0.0 %
Customers,0,0.0 %
Open,0,0.0 %
Promo,0,0.0 %


In [7]:
combined_df['Promo2SinceWeek'] = combined_df['Promo2SinceWeek'].fillna(
    combined_df['Promo2SinceWeek'].max())
combined_df['Promo2SinceYear'] = combined_df['Promo2SinceYear'].fillna(
    combined_df['Promo2SinceYear'].max())
combined_df['CompetitionDistance'] = combined_df['CompetitionDistance'].fillna(
    combined_df['CompetitionDistance'].max())

combined_df['PromoInterval'] = combined_df['PromoInterval'].fillna(
    combined_df['PromoInterval'].mode().iloc[0])
combined_df['CompetitionOpenSinceMonth'] = combined_df['CompetitionOpenSinceMonth'].fillna(
    combined_df['CompetitionOpenSinceMonth'].mode().iloc[0])
combined_df['CompetitionOpenSinceYear'] = combined_df['CompetitionOpenSinceYear'].fillna(
    combined_df['CompetitionOpenSinceYear'].mode().iloc[0])

In [8]:
info.get_columns_with_missing_values()

[]

## Adding Additional Extracted Data

### WeekDays

In [9]:
def add_week_day(dataframe:pd.DataFrame, day_of_week_col:str) -> pd.DataFrame:
    date_index = dataframe.columns.get_loc(day_of_week_col)
    dataframe = dataframe.copy(deep=True)
    dataframe.insert(date_index + 1, 'WeekDay', dataframe[day_of_week_col].apply(lambda x: 1 if x <= 5 else 0))
    
    return dataframe

In [10]:
combined_df = add_week_day(combined_df, 'DayOfWeek')

### WeekEnds


> Doesnt Add additional information

In [11]:
# def add_week_ends(dataframe: pd.DataFrame, day_of_week_col: str) -> pd.DataFrame:
#     date_index = dataframe.columns.get_loc(day_of_week_col)
#     dataframe = dataframe.copy(deep=True)
#     dataframe.insert(date_index + 1, 'WeekEnd',
#                      dataframe[day_of_week_col].apply(lambda x: 1 if x > 5 else 0))

#     return dataframe

In [12]:
# combined_df = add_week_(combined_df, 'DayOfWeek')


### Number of days to Holidays


In [13]:
# Considering christmas lasts for 12 days, Easter for 50 days and public holidays for 1 day.
# And considering before and after periods to be 5 less and 5 more days before and after the holiday for christmas
# and 10 days for Easter
# And 3 days for public holiday
# get state holiday list
#a = public holiday, b = Easter holiday, c = Christmas, 0 = None
def affect_list(change_list, interval, duration, index):
    start_pt = int(index-duration/2) - interval
    try:
        for index in range(start_pt, start_pt + interval):
            change_list[index] = 'before'
        for index in range(start_pt + interval, start_pt + interval + duration):
            change_list[index] = 'during'
        for index in range(start_pt + interval + duration, start_pt + interval + duration + interval):
            change_list[index] = 'after'
    except:
        pass

    return change_list

# state_holiday_list = train_df['StateHoliday'].values.tolist()

def modify_holiday_list(holiday_list:list) -> list:
    new_index = ["neither"] * len(holiday_list)
    for index , value in enumerate(holiday_list):
        if value == 'a': #public holiday
            affect_list(new_index, 3, 1, index)
        elif value == 'b': #Easter
            affect_list(new_index, 10, 50, index)
        elif value == 'c': # christmas
            affect_list(new_index, 5, 12, index)
        else:
            pass

    return new_index

def add_number_of_days_to_holiday(dataframe:pd.DataFrame, state_holiday_col:str):
    date_index = dataframe.columns.get_loc(state_holiday_col)
    dataframe = dataframe.copy(deep=True)

    modified_index = modify_holiday_list(dataframe[state_holiday_col].values.tolist())
    days_to_holiday_index = []
    i = 0
    last_holiday_index = 0
    for index, value in enumerate(modified_index):
        if(index == len(modified_index) - 1):
            for j in range(last_holiday_index+1,len(modified_index)):
                days_to_holiday_index.append(0)
        elif(value == 'neither' or value == 'after' or value == 'before'):
            i += 1
        elif(value == 'during' and i != 0):
            last_holiday_index = index
            for j in range(i):
                days_to_holiday_index.append(i)
                i = i-1
            days_to_holiday_index.append(0)
            i = 0
        elif(value == 'during' and i == 0):
            days_to_holiday_index.append(i)
            last_holiday_index = index
            continue

    dataframe.insert(date_index + 1, 'DaysToHoliday',
                     days_to_holiday_index)

    return dataframe

In [14]:
combined_df = add_number_of_days_to_holiday(combined_df, 'StateHoliday')


### Number of days after Holiday


In [15]:
def add_number_of_days_after_holiday(dataframe: pd.DataFrame, state_holiday_col: str):
    date_index = dataframe.columns.get_loc(state_holiday_col)
    dataframe = dataframe.copy(deep=True)

    modified_index = modify_holiday_list(
        dataframe[state_holiday_col].values.tolist())

    days_to_after_holiday_index = [0] * len(modified_index)
    i = 0
    last_holiday_index = modified_index.index('during')

    for index, value in enumerate(modified_index):
        if(value == 'before'):
            if(index > last_holiday_index):
                i += 1
                days_to_after_holiday_index[index] = i
            continue
        elif(value == 'after'):
            i += 1
            days_to_after_holiday_index[index] = i
        elif(value == 'during'):
            last_holiday_index = index
            i = 0
            continue

    days_to_after_holiday_index.insert(0,0)

    dataframe.insert(date_index + 1, 'DaysAfterHoliday',
                     days_to_after_holiday_index[:-1])

    return dataframe


In [16]:
combined_df = add_number_of_days_after_holiday(combined_df, 'StateHoliday')


### Beginning of month, mid month and ending of month


In [17]:
def return_day_status_in_month(day:int) -> int:
    ## conside 1 is beginning of month, 2 is middle of the month and 3 is end of the month 
    if(day <= 10):
        return 1
    elif(day > 10 and day <= 20):
        return 2
    else:
        return 3


def add_month_timing(dataframe: pd.DataFrame, day_col: str) -> pd.DataFrame:
    date_index = dataframe.columns.get_loc(day_col)
    dataframe = dataframe.copy(deep=True)
    dataframe.insert(date_index + 1, 'MonthTiming', dataframe[day_col].apply(return_day_status_in_month))

    return dataframe

In [18]:
combined_df = add_month_timing(combined_df, 'Day')


### More Features

### Season (Winter, April, ...)

In [19]:
def get_season(month: int):
            if(month <= 2 or month == 12):
                return 'Winter'
            elif(month > 2 and month <= 5):
                return 'Spring'
            elif(month > 5 and month <= 8):
                return 'Summer'
            else:
                return 'Autumn'


def add_season(dataframe: pd.DataFrame, month_col: str) -> pd.DataFrame:
    date_index = dataframe.columns.get_loc(month_col)
    dataframe = dataframe.copy(deep=True)
    dataframe.insert(date_index + 1, 'Season',
                     dataframe[month_col].apply(get_season))

    return dataframe

In [20]:
combined_df = add_season(combined_df, 'Month')


In [21]:
## Added dataframe status
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 26 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  uint16 
 1   DayOfWeek                  1017209 non-null  uint8  
 2   WeekDay                    1017209 non-null  int64  
 3   Date                       1017209 non-null  object 
 4   Year                       1017209 non-null  uint16 
 5   Month                      1017209 non-null  uint8  
 6   Season                     1017209 non-null  object 
 7   Day                        1017209 non-null  uint8  
 8   MonthTiming                1017209 non-null  int64  
 9   Sales                      1017209 non-null  uint16 
 10  Customers                  1017209 non-null  uint16 
 11  Open                       1017209 non-null  uint8  
 12  Promo                      1017209 non-null  uint8  
 13  StateHoliday

## Drop Columns

In [22]:
# Store	Date	Customers
combined_df.drop(['Store','Date','Customers'],axis=1,inplace=True)

In [23]:
## Dropped dataframe status
combined_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DayOfWeek                  1017209 non-null  uint8  
 1   WeekDay                    1017209 non-null  int64  
 2   Year                       1017209 non-null  uint16 
 3   Month                      1017209 non-null  uint8  
 4   Season                     1017209 non-null  object 
 5   Day                        1017209 non-null  uint8  
 6   MonthTiming                1017209 non-null  int64  
 7   Sales                      1017209 non-null  uint16 
 8   Open                       1017209 non-null  uint8  
 9   Promo                      1017209 non-null  uint8  
 10  StateHoliday               1017209 non-null  object 
 11  DaysAfterHoliday           1017209 non-null  int64  
 12  DaysToHoliday              1017209 non-null  int64  
 13  SchoolHolida

## Encoding Data to Numbers

In [24]:
# StoreType	Assortment StateHoliday Season
le = preprocessing.LabelEncoder()
season_le = le.fit(combined_df['Season'].values)
combined_df['Season'] = season_le.transform(combined_df['Season'].values)

stateholiday_le = le.fit(combined_df['StateHoliday'].values)
combined_df['StateHoliday'] = stateholiday_le.transform(combined_df['StateHoliday'].values)

storetype_le = le.fit(combined_df['StoreType'].values)
combined_df['StoreType'] = storetype_le.transform(combined_df['StoreType'].values)

assortment_le = le.fit(combined_df['Assortment'].values)
combined_df['Assortment'] = assortment_le.transform(combined_df['Assortment'].values)


In [25]:
combined_df['PromoInterval'].value_counts()

Jan,Apr,Jul,Oct     801153
Feb,May,Aug,Nov     118596
Mar,Jun,Sept,Dec     97460
Name: PromoInterval, dtype: int64

In [26]:
promointerval_le = le.fit(combined_df['PromoInterval'].values)
combined_df['PromoInterval'] = promointerval_le.transform(combined_df['PromoInterval'].values)


In [27]:
## Label Encoded dataframe status
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 23 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DayOfWeek                  1017209 non-null  uint8  
 1   WeekDay                    1017209 non-null  int64  
 2   Year                       1017209 non-null  uint16 
 3   Month                      1017209 non-null  uint8  
 4   Season                     1017209 non-null  int32  
 5   Day                        1017209 non-null  uint8  
 6   MonthTiming                1017209 non-null  int64  
 7   Sales                      1017209 non-null  uint16 
 8   Open                       1017209 non-null  uint8  
 9   Promo                      1017209 non-null  uint8  
 10  StateHoliday               1017209 non-null  int32  
 11  DaysAfterHoliday           1017209 non-null  int64  
 12  DaysToHoliday              1017209 non-null  int64  
 13  SchoolHolida

## Scaling Data

In [28]:
# Using StandardScaler to standardize the all columns
manipulator = DataManipulator(combined_df)
manipulator.standardize_columns(combined_df.columns.to_list())
info = DataInfo(manipulator.df)
info.get_min_max_of_dataframe_columns()

,Max Value,Min Value
DayOfWeek,1.502791,-1.501129
WeekDay,0.630672,-1.585611
Year,1.502077,-1.070616
Month,1.849989,-1.457193
Season,1.333709,-1.498283
Day,1.740766,-1.673123
MonthTiming,1.205894,-1.236292
Sales,9.292957,-1.499723
Open,0.452399,-2.210440
Promo,1.273237,-0.785400


# Save Clean Data

In [30]:
cleaner = DataCleaner(info.df)
cleaner.optimize_df()
cleaner.save_clean_data('../data/train.csv')